In [1]:
import lzma
import pickle

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from neural_network import NeuralNetwork

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Load and preprocess your collected data


In [3]:
snapshots = []

with lzma.open("../record_0.npz", "rb") as file:
    snapshots = pickle.load(file)

snapshots[0:5]

package_folder: /home/nathan/projects/RallyRobotPilot_2025/.venv/lib/python3.13/site-packages/ursina
asset_folder: /home/nathan/projects/RallyRobotPilot_2025/.venv/lib/python3.13/site-packages


which: no blender in (/home/nathan/projects/RallyRobotPilot_2025/.venv/bin:/home/nathan/.rvm/gems/ruby-3.3.8/bin:/home/nathan/.rvm/gems/ruby-3.3.8@global/bin:/home/nathan/.rvm/rubies/ruby-3.3.8/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/sbin:/usr/lib/jvm/default/bin:/usr/bin/site_perl:/usr/bin/vendor_perl:/usr/bin/core_perl:/home/nathan/.rvm/bin)


In [10]:
X_scalar_list = [
    [s.car_speed, *s.raycast_distances]
    for s in snapshots
]
X_image_list = [s.image for s in snapshots]

# 2. Create the list of labels (y). The original list comprehension was a bit complex.
y_list = [s.current_controls for s in snapshots]

# 3. Convert all lists to NumPy arrays with a specific data type
features = np.array(X_scalar_list, dtype=np.float32)
labels = np.array(y_list, dtype=np.float32)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define your neural network model


In [5]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [19]:
features = torch.rand(4, 16, device=device)
logits = model(features)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3, 3, 3, 3])


# Set up training loop with loss function and optimizer

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-4)

# Train the model and save checkpoints


In [ ]:
from torch.utils.data import Dataset

class Data(Dataset):
    def __init__(self, X, y):
        self.features = torch.from_numpy(X.astype(np.float32))
        self.labels = torch.from_numpy(y.astype(np.float32))  # Changed from int32 to float32
        self.len = self.features.shape[0]
       
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
   
    def __len__(self):
        return self.len

In [12]:
batch_size = 32

train_data = Data(X_train, y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [ ]:
num_epochs = 100
loss_values = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for features, labels in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
       
        # forward + backward + optimize
        pred = model(features)
        loss = criterion(pred, labels)  # Removed unsqueeze(-1)
        total_loss += loss.item()
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

ValueError: Using a target size (torch.Size([32, 4, 1])) that is different to the input size (torch.Size([32, 4])) is deprecated. Please ensure they have the same size.

# Evaluate performance on validation data
